# Chapter 17 Textbook: Classification


**Reading**: 
* [Classification](https://data8-demo.jupyterlite.app/chapters/17/classification)

Please complete this notebook by filling in the cells provided.

Directly sharing answers is not okay, but discussing problems with the course staff or with other students is encouraged.

For all problems that you must write our explanations and sentences for, you must provide your answer in the designated space. Moreover, throughout this homework and all future ones, please be sure to not re-assign variables throughout the notebook! For example, if you use max_temperature in your answer to one question, do not reassign it later on.

In [ ]:
# Don't change this cell; just run it. 
import numpy as np
from datascience import *

In [ ]:
# These lines do some fancy plotting magic.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)

In [ ]:
def standard_units(arr):
    return (arr - np.average(arr))/np.std(arr)

In [ ]:
ckd = Table.read_table('ckd.csv').relabeled('Blood Glucose Random', 'Glucose')
ckd.row(0)

In [ ]:
ckd = Table().with_columns(
    'Hemoglobin', standard_units(ckd.column('Hemoglobin')),
    'Glucose', standard_units(ckd.column('Glucose')),
    'White Blood Cell Count', standard_units(ckd.column('White Blood Cell Count')),
    'Class', ckd.column('Class')
)
ckd

In [ ]:
color_table = Table().with_columns(
    'Class', make_array(1, 0),
    'Color', make_array('darkblue', 'gold')
)
ckd = ckd.join('Class', color_table)

In [ ]:
ckd

In [ ]:
ckd.scatter('White Blood Cell Count', 'Glucose', group='Color')
plt.xlim(-4, 2)
plt.ylim(-2, 4);

In [ ]:
ckd.scatter('Hemoglobin', 'Glucose', group='Color')
plt.xlim(-4, 2)
plt.ylim(-2, 4);

In [ ]:
shuffled_ckd = ckd.sample(with_replacement=False)
training = shuffled_ckd.take(np.arange(79))
testing = shuffled_ckd.take(np.arange(79, 158))

In [ ]:
training

In [ ]:
training.scatter('Hemoglobin', 'Glucose', group='Color')
plt.xlim(-4, 2)
plt.ylim(-2, 4);

In [ ]:
testing.scatter('Hemoglobin', 'Glucose', group='Color')
plt.xlim(-4, 2)
plt.ylim(-2, 4);

In [ ]:
alice = make_array(0, 1.1)

training.scatter('Hemoglobin', 'Glucose', group='Color')
plt.xlim(-4, 2)
plt.ylim(-2, 4)
plt.scatter(alice.item(0), alice.item(1), color='red', s=30);

In [ ]:
training_set= training.select('Class','Hemoglobin','Glucose')
test_set = testing.select('Class','Hemoglobin','Glucose')

In [ ]:
training_set

In [ ]:
test_set

In [ ]:
training_attributes = training.select('Hemoglobin', 'Glucose')
training_attributes

In [ ]:
alice

In [ ]:
patient3 = np.array(training_attributes.row(3))
patient3

In [ ]:
def distance(point1, point2):
    """Returns the distance between point1 and point2
    where each argument is an array 
    consisting of the coordinates of the point"""
    return np.sqrt(np.sum((point1 - point2)**2))

def all_distances(training, new_point):
    """Returns an array of distances
    between each point in the training set
    and the new point (which is a row of attributes)"""
    attributes = training.drop('Class')
    def distance_from_point(row):
        return distance(np.array(new_point), np.array(row))
    return attributes.apply(distance_from_point)

def table_with_distances(training, new_point):
    """Augments the training table 
    with a column of distances from new_point"""
    return training.with_column('Distance', all_distances(training, new_point))

def closest(training, new_point, k):
    """Returns a table of the k rows of the augmented table
    corresponding to the k smallest distances"""
    with_dists = table_with_distances(training, new_point)
    sorted_by_distance = with_dists.sort('Distance')
    topk = sorted_by_distance.take(np.arange(k))
    return topk

def majority(topkclasses):
    ones = topkclasses.where('Class', are.equal_to(1)).num_rows
    zeros = topkclasses.where('Class', are.equal_to(0)).num_rows
    if ones > zeros:
        return 1
    else:
        return 0

def classify(training, new_point, k):
    closestk = closest(training, new_point, k)
    topkclasses = closestk.select('Class')
    return majority(topkclasses)

In [ ]:
distance(alice,patient3)

In [ ]:
closest(training_attributes, alice, 5)

In [ ]:
closest(training_attributes, patient3, 5)

In [ ]:
def count_zero(array):
    """Counts the number of 0's in an array"""
    return len(array) - np.count_nonzero(array)

def count_equal(array1, array2):
    """Takes two numerical arrays of equal length
    and counts the indices where the two are equal"""
    return count_zero(array1 - array2)

def evaluate_accuracy(training, test, k):
    test_attributes = test.drop('Class')
    def classify_testrow(row):
        return classify(training, row, k)
    c = test_attributes.apply(classify_testrow)
    return count_equal(c, test.column('Class')) / test.num_rows

In [ ]:
evaluate_accuracy(training_set, test_set, 5)